## Note: All visualizations are interactive so you may hover on them to get more information and even zoom, pan etc. 

## Some visualizations will appear to be repeated if the notebook is not run. Please run each cell to get the appropriate visualization



# Gene related distributions

In [44]:
import pandas as pd

df = pd.read_excel("pr1.xlsx", sheet_name='Variant_Details')

# Check the total number of unique genes in the 'GENE' column
unique_genes = df['GENE'].nunique()
print(f'Total number of unique genes: {unique_genes}')


Total number of unique genes: 738


In [45]:
import pandas as pd
import plotly.express as px


# Get the top 10 genes with the highest occurrences
top_genes = df['GENE'].value_counts().head(10)
top_genes_df = df[df['GENE'].isin(top_genes.index)]

# Create a new DataFrame to store the patient information for each gene
hover_data = top_genes_df.groupby('GENE')['Contact_ID'].apply(list).reset_index()
hover_data['Contact_ID'] = hover_data['Contact_ID'].apply(lambda x: ', '.join(x))

# Merge the data for plotting
plot_data = top_genes.reset_index()
plot_data = plot_data.merge(hover_data, left_on='index', right_on='GENE')

# Create the interactive bar chart
fig = px.bar(plot_data, x='index', y='GENE_x', text='GENE_x', hover_name='Contact_ID', title='Top 10 Genes with Highest Occurrences')
fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(xaxis_title='Gene', yaxis_title='Frequency', showlegend=False)
fig.show()
fig.write_html("top 10 genes.html")

x-axis: top 10 genes ordered by the highest number of patients having them (occurrences)

y-axis: number of patients having that gene 

Interpretation: 1910 patients have PCLO gene, 1641 patients have SZT2 gene and so on...

In [46]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd

# Sample DataFrame (replace this with loading your actual data)


# Initialize the Dash app
app = Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    dcc.Input(id='gene-input', type='text', placeholder='Enter a gene'),
    html.Button('Submit', id='submit-button', n_clicks=0),
    dcc.Graph(id='gene-bar-plot'),
    html.Div(id='patient-list')
])

# Define the callback to update the graph and display patient list
@app.callback(
    [Output('gene-bar-plot', 'figure'),
     Output('patient-list', 'children')],
    [Input('submit-button', 'n_clicks')],
    [Input('gene-input', 'value')]
)
def update_graph(n_clicks, gene_input):
    if n_clicks > 0 and gene_input:
        filtered_df = df[df['GENE'] == gene_input]
        gene_count = len(filtered_df)
        patients = filtered_df['Contact_ID'].tolist()
        
        # Create bar plot
        fig = go.Figure(data=[go.Bar(x=[gene_input], y=[gene_count], text=[gene_count])])
        fig.update_traces(texttemplate='%{text}', textposition='outside')
        fig.update_layout(xaxis_title='Gene', yaxis_title='Count', showlegend=False)
        
        # Create patient list
        patient_list = html.Ul([html.Li(patient) for patient in patients])
        
        return fig, patient_list
    return go.Figure(), None

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


The above visualization is very useful if you want to know the number of patients having a particular gene along with the patient id of those patients

Let's say I want to know how many patients have PCLO so I just go and search PCLO. Along with the count of the patients who have PLCO, all the patients having PCLO are displayed. This a particularly useful visualization in knowing and accessing entries of patients having a particular gene 

In [26]:
# Function to update the plot based on the selected gene
import pandas as pd
import plotly.express as px
from ipywidgets import widgets
from IPython.display import display

df_variant_details = pd.read_excel("pr1.xlsx", sheet_name='Variant_Details')
df_patient_results = pd.read_excel("pr1.xlsx", sheet_name='Patient_Results')

def plot_gene_distribution(gene):
    # Merge dataframes
    df_merged = pd.merge(df_patient_results, df_variant_details, on='Contact_ID')
    
    # Filter based on selected gene
    df_filtered = df_merged[df_merged['GENE'] == gene]
    
    # Group by state and count occurrences
    state_counts = df_filtered['Institution_ST'].value_counts().reset_index()
    state_counts.columns = ['State', 'Count']
    
    # Plot
    fig = px.choropleth(state_counts, 
                        locations='State', 
                        color='Count',
                        locationmode="USA-states", 
                        scope="usa",
                        title=f'Distribution of {gene} Across States')
    fig.show()

# Create widget
gene_widget = widgets.Combobox(
    options=df_variant_details['GENE'].unique().tolist(),
    value='SAMHD1',
    description='Select Gene:',
)

# Display widget
widgets.interactive(plot_gene_distribution, gene=gene_widget)

interactive(children=(Combobox(value='SAMHD1', description='Select Gene:', options=('SAMHD1', 'KCNQ2', 'ADAR',…

'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'

The above interactive visualization allows you to search a gene (or you could choose one from the dropdown) and then you are given the count and distribution of that particular gene across every state. You can hover on a state to get the number patients who have that gene in that particular state. This plot allows you to see the trends of each gene across different states 

Interesting to see how Texas is always the state having the most number of patients for any given gene

In [27]:
#Zygosity

import ipywidgets as widgets
import plotly.express as px
import pandas as pd

# Calculate the frequency of each category in 'ZYGOSITY'
zygosity_counts = df['ZYGOSITY'].value_counts()
unique_zygosities = df['ZYGOSITY'].nunique()

# Display the number of unique values
zygosity_text = widgets.HTML(value=f"<b>Number of Unique Zygosity Values:</b> {unique_zygosities}")
display(zygosity_text)

# Create the interactive bar chart
fig = px.bar(zygosity_counts, title='Frequency of Each Zygosity Category')
fig.update_layout(xaxis_title='Zygosity', yaxis_title='Frequency', showlegend=False)
fig.show()


HTML(value='<b>Number of Unique Zygosity Values:</b> 10')

Zygosity: Refers to the genetic state of an individual with respect to a particular variant.
    Types:
        Homozygous: Two copies of the variant are present (one from each parent).
        Heterozygous: Only one copy of the variant is present.
        Hemizygous: The variant is present on a sex chromosome in males, where there is no corresponding allele.

In [28]:
import plotly.graph_objects as go
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import pandas as pd

# Read the Excel file
df = pd.read_excel('pr1.xlsx', sheet_name = 'Variant_Details')

# Initialize the Dash app
app = Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='interpretation-type',
        options=[
            {'label': 'Original Interpretation', 'value': 'ORIGINAL_INTERPRETATION'},
            {'label': 'Revised Interpretation', 'value': 'REVISED_INTERPRETATION'}
        ],
        value='ORIGINAL_INTERPRETATION',
        multi=False
    ),
    dcc.Graph(id='bar-plot')
])

# Define the callback to update the bar plot
@app.callback(
    Output('bar-plot', 'figure'),
    [Input('interpretation-type', 'value')]
)
def update_bar_plot(selected_interpretation):
    # Filter the data based on the selected interpretation type
    interpretation_data = df[selected_interpretation].value_counts().reset_index()
    interpretation_data.columns = ['Category', 'Count']
    
    # Create the bar plot
    fig = go.Figure(go.Bar(
        x=interpretation_data['Count'],
        y=interpretation_data['Category'],
        orientation='h'
    ))
    fig.update_layout(
        title=f'Distribution of {selected_interpretation.replace("_", " ")}',
        xaxis_title='Count',
        yaxis_title='Category'
    )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


The above graph shows the distribution of different categories in the original interpretation and "Revised interpretation columns. You can use the dropdown to toggle which one you want to see

In [29]:
#Heatmap showing how the revised interpretation was different from the original interpretation

To undertand how the revised interpretation changed from the original interpretation, this heatmap was plotted. 

Interpretation: 

- 78793 cells/rows which had "Uncertain Significance" as the original interpreation changed to "Not Provided"
- 9559 cells/rows which had "Pathogenic" as the original interpretation changed to "Not Provided" 
- 267 cells/rows which had "Likely Pathogenic" as the original interpreation changed to "Pathogenic" in the Revised Interpretation

More inferences can be drawn in a similar manner

We can also use both the above plots to make even more detailed inferences. For instance: 

- The total number of rows having likely pathogenic in original interpretation was 1701 but only 267 rows changed to pathogenic
- The total number of rows having pathogenic was 9567 in original interpretation but 9559 changed to not provided. Too much missing data!
- There seems to be a trend where a majority of them end up in the "Not provided" category!

# Clinician related distributions

In [30]:
import plotly.express as px
import pandas as pd

# Creating a DataFrame from the given data
data = {
    'Clinician': ['Atiya Khan', 'Shalaka Indulkar', 'Rebecca Luke', 'Mark Fitzgerald', 
                  'Jesus Pina-Garza', 'Bernardo Flasterstein', 'Jasna Kojic', 'Naeem Mahfooz', 
                  'Melissa Assaf', 'Peter Castellanos'],
    'Patients': [425, 260, 216, 215, 195, 191, 176, 175, 157, 156]
}

df = pd.DataFrame(data)

# Sorting the DataFrame based on the number of patients
df = df.sort_values(by='Patients', ascending=False)

# Creating the bar chart
fig = px.bar(df, x='Clinician', y='Patients', title='Top 10 Clinicians Based on Number of Patients Seen',
             labels={'Patients': 'Number of Patients'}, text='Patients')

# Showing the plot
fig.show()


In [31]:

import pandas as pd
import plotly.express as px

clinician_data = pd.read_excel("pr1.xlsx", sheet_name='Accessioned_By_Clinician')

import pandas as pd
import plotly.express as px

# Data Preparation
date_columns = clinician_data.columns[6:-2]
data_transposed = clinician_data.melt(id_vars='Clinician', value_vars=date_columns, var_name='Date', value_name='Patients')
data_transposed['Date'] = pd.to_datetime(data_transposed['Date'])
data_grouped = data_transposed.groupby(['Clinician', 'Date'])['Patients'].sum().reset_index()

# Identify Top Clinicians
top_clinicians = data_grouped.groupby('Clinician')['Patients'].sum().sort_values(ascending=False).head(10)

# Time Series Analysis
data_top_clinicians = data_grouped[data_grouped['Clinician'].isin(top_clinicians.index)]
data_top_clinicians['Year'] = data_top_clinicians['Date'].dt.year
data_top_clinicians['Quarter'] = data_top_clinicians['Date'].dt.to_period("Q").astype(str)
data_grouped_top_clinicians = data_top_clinicians.groupby(['Clinician', 'Year', 'Quarter'])['Patients'].sum().reset_index()

# Create an interactive line plot
fig = px.line(data_grouped_top_clinicians, x='Quarter', y='Patients', color='Clinician', 
              title='Trend of Number of Patients Seen by Top 10 Clinicians Over Each Quarter',
              labels={'Patients': 'Number of Patients', 'Quarter': 'Quarter'},
              template='plotly_white')

# Show the plot
fig.show()


/var/folders/l4/3ctxtv0s4f10lcsxz1mr2_080000gn/T/ipykernel_58849/3598726933.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/l4/3ctxtv0s4f10lcsxz1mr2_080000gn/T/ipykernel_58849/3598726933.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



You can click on the clinicians on the right to add or remove the particular clinician from the graph and can hover at any point to get more info

- Interesting to note how Atiya Khan is consistently seeing more patients than any other clinician. Perhaps Atiya is a renowned clinican in the field?

In [32]:
import pandas as pd
import plotly.express as px

# Read data
df_institute = pd.read_excel('pr1.xlsx', sheet_name='Accessioned_By_Institute')
df_clinician = pd.read_excel('pr1.xlsx', sheet_name='Accessioned_By_Clinician')

# Calculate total patients for each institute and clinician
df_institute['Total_Patients'] = df_institute.iloc[:, 5:].sum(axis=1)
df_clinician['Total_Patients'] = df_clinician.iloc[:, 6:].sum(axis=1)

# Get top 20 institutes
top_institutes = df_institute.nlargest(20, 'Total_Patients')

# Initialize an empty DataFrame to store top 5 clinicians for each top institute
top_clinicians = pd.DataFrame()

# For each top institute, find top 5 clinicians
for index, row in top_institutes.iterrows():
    institute_id = row['Institution ID']
    clinicians = df_clinician[df_clinician['Institution ID'] == institute_id]
    top_clinicians_in_institute = clinicians.nlargest(5, 'Total_Patients')
    top_clinicians = pd.concat([top_clinicians, top_clinicians_in_institute])

# Create an interactive bar chart
fig = px.bar(top_clinicians, x='Institution', y='Total_Patients', color='Clinician', title='Top 5 Clinicians in Top 20 Institutes')

# Show the plot
fig.show()


# Age groups related trends

In [33]:
df = pd.read_excel("pr1.xlsx", sheet_name='Aggregate_Volumes_by_Age')

# Set the 'Age_Group' column as the index
df.set_index('Age_Group', inplace=True)

# Transpose the DataFrame to have dates as columns and age groups as rows
df = df.transpose()

# Convert the index to datetime objects
df.index = pd.to_datetime(df.index)

# Create an interactive time series plot using Plotly
fig = px.line(df, x=df.index, y=df.columns, title='Time Series Analysis: Trend Over Time for Each Age Group')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Count')

# Show the interactive plot
fig.show()

- Number of patients across each age group seems to be increasing overall. Maybe indicates more awareness among the public or a natural tendancy because of population increase

# Institutional based trends 

In [34]:
import pandas as pd
import plotly.express as px


df = pd.read_excel("pr1.xlsx", sheet_name="Accessioned_By_Institute")

# Calculate total number of patients for each institute
df['Total Patients'] = df.iloc[:, 5:].sum(axis=1)

# Sort institutes by total patients in descending order and select top 20
top_institutes = df.sort_values(by='Total Patients', ascending=False).head(20)

# Create interactive bar chart
fig = px.bar(top_institutes, x='Institution', y='Total Patients', title='Top 20 Institutes by Number of Patients')
fig.show()


In [35]:
# Read the data from Sheet2
# Load the Excel file
file_path = 'pr1.xlsx'
excel_data = pd.ExcelFile(file_path)
# Read the data from Sheet1
sheet1_data = excel_data.parse('Accessioned_By_Institute')

sheet2_data = excel_data.parse('Accessioned_By_Clinician')

# Select the top 5 clinicians based on the number of patients (Grand_total)
top_clinicians = sheet2_data.nlargest(5, 'Grand_total')

# Extracting the top 5 clinicians
top_clinician_names = top_clinicians['Clinician'].unique()

# Filtering the data to include only the top 5 clinicians
top_clinicians_data = sheet2_data[sheet2_data['Clinician'].isin(top_clinician_names)]

# Grouping the data by clinician and aggregating the institution values
clinician_institutions = top_clinicians_data.groupby('Clinician')['Institution'].unique().reset_index()

# Grouping the data by institution and counting the number of unique clinicians in each institution
institution_clinician_count = sheet2_data.groupby('Institution')['Clinician'].nunique().sort_values(ascending=False)

# Selecting the top 20 institutions
top_20_institutions = institution_clinician_count.head(20)

# Creating an interactive bar chart
fig = px.bar(
    top_20_institutions,
    x=top_20_institutions.index,
    y=top_20_institutions.values,
    title='Top 20 Institutions with the Most Number of Clinicians',
    labels={'x': 'Institution', 'y': 'Number of Clinicians'},
    template='plotly_white'
)

# Updating the layout for better readability
fig.update_layout(xaxis_title='Institution', yaxis_title='Number of Clinicians', xaxis_tickangle=-45)
fig.update_traces(marker_color='skyblue')

# Showing the plot
fig.show()


The above visualizations shows the top 20 institues by the number of clinicians that they have

Note: There are clinicians who visit multiple institutes

- The above 2 visualizations when seen together help in seeing whether the insitutes with a given number of patients have how many clinician and vice versa

In [36]:
# Load the Excel file
file_path = 'pr1.xlsx'
excel_data = pd.ExcelFile(file_path)
# Read the data from Sheet1
sheet1_data = excel_data.parse('Accessioned_By_Institute')

# Select the top 10 institutions based on the number of patients (Grand_total)
top_institutions = sheet1_data.nlargest(10, 'Grand_total')

# Selecting only the weekly patient data for the top 10 institutions
weekly_data = top_institutions.iloc[:, 5:-3].T
weekly_data.columns = top_institutions['Institution']

# Converting the index to datetime
weekly_data.index = pd.to_datetime(weekly_data.index)

# Resampling the data to get the sum for each quarter
quarterly_data = weekly_data.resample('Q').sum()

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Create a subplot
fig = make_subplots(rows=1, cols=1)

# Define a colormap
colormap = px.colors.sequential.Viridis

# Add traces for each institution
for i, institution in enumerate(quarterly_data.columns):
    color = colormap[int(i * len(colormap) / len(quarterly_data.columns))]
    fig.add_trace(go.Scatter(x=quarterly_data.index, y=quarterly_data[institution], 
                             mode='lines+markers', name=institution, 
                             line=dict(color=color)))

# Update the layout
fig.update_layout(title_text='Time Series Patient Trend for Top 10 Institutions',
                  xaxis_title='Date',
                  yaxis_title='Number of Patients',
                  legend_title='Institution',
                  template='plotly')

# Show the plot
fig.show()


The above visualization gives us a trend of how the number of patients are changing over time at the top 10 institutes with the most number of patients

You can click on the institution on the right to add or remove it from the visualization 

# Patient Demographic based trends

In [37]:
import plotly.express as px
import pandas as pd

data_sheet2 = pd.read_excel("pr1.xlsx", sheet_name='Accessioned_By_Clinician')
# Aggregating the total number of patients for each country
country_total_patients = data_sheet2.groupby('Country')['Grand_total'].sum().sort_values(ascending=False).reset_index()

# Visualization using Plotly
fig = px.bar(country_total_patients, x='Country', y='Grand_total', title='Distribution of Patients Across Countries')
fig.update_layout(xaxis_title='Country', yaxis_title='Total Number of Patients', autosize=False, width=1000, height=500)
fig.show()

In [41]:
df = pd.read_excel("pr1.xlsx", sheet_name='Patient_Results')

# Group by 'Accession_year' and count the number of patients in each year
patient_distribution = df['Accession_year'].value_counts().reset_index()
patient_distribution.columns = ['Year', 'Number of Patients']

# Sort the dataframe based on 'Year'
patient_distribution = patient_distribution.sort_values(by='Year')

# Create interactive bar plot
fig = px.bar(patient_distribution, x='Year', y='Number of Patients', title='Distribution of Patients Based on Year of Accession')
fig.show()


There is a steady increase in the number of patients being accessioned in every year. We don't have the data for 2023 yet but we can say that 2023 would conform to the trend and end up being highest

In [42]:
# Group by 'Institution_City' and count the number of patients in each city
patient_distribution = df['Institution_City'].value_counts().reset_index()
patient_distribution.columns = ['City', 'Number of Patients']

# Initialize the Dash app
app = Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='num-top-cities-dropdown',
        options=[{'label': f'Top {i}', 'value': i} for i in range(1, len(patient_distribution) + 1)] + 
                 [{'label': f'Bottom {i}', 'value': -i} for i in range(1, len(patient_distribution) + 1)],
        value=20,
        clearable=False,
        searchable=True,
        style={'width': '50%'}
    ),
    dcc.Graph(id='patient-distribution-bar-plot')
])

# Define the callback to update the graph based on the selected number of top cities
@app.callback(
    Output('patient-distribution-bar-plot', 'figure'),
    Input('num-top-cities-dropdown', 'value')
)
def update_graph(num_top_cities):
    # Filter the dataframe to get the top N or bottom N cities
    if num_top_cities > 0:
        filtered_cities = patient_distribution.head(num_top_cities)
    else:
        filtered_cities = patient_distribution.tail(-num_top_cities)
    
    # Create interactive bar plot
    fig = px.bar(filtered_cities, x='City', y='Number of Patients', title='Distribution of Patients Based on City')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


You can choose to display the number of cities you want to display. If you choose the top 10 it will give you the top 10. You also have the option to select bottom 10

In [40]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# Group by 'Institution_ST' and count the number of patients in each state
patient_distribution = df['Institution_ST'].value_counts().reset_index()
patient_distribution.columns = ['State', 'Number of Patients']

# Initialize the Dash app
app = Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='num-top-states-dropdown',
        options=[{'label': f'Top {i}', 'value': i} for i in range(1, len(patient_distribution) + 1)] + 
                 [{'label': f'Bottom {i}', 'value': -i} for i in range(1, len(patient_distribution) + 1)],
        value=20,
        clearable=False,
        searchable=True,
        style={'width': '50%'}
    ),
    dcc.Graph(id='patient-distribution-bar-plot')
])

# Define the callback to update the graph based on the selected number of top states
@app.callback(
    Output('patient-distribution-bar-plot', 'figure'),
    Input('num-top-states-dropdown', 'value')
)
def update_graph(num_top_states):
    # Filter the dataframe to get the top N or bottom N states
    if num_top_states > 0:
        filtered_states = patient_distribution.head(num_top_states)
    else:
        filtered_states = patient_distribution.tail(-num_top_states)
    
    # Create interactive bar plot
    fig = px.bar(filtered_states, x='State', y='Number of Patients', title='Distribution of Patients Based on State')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [19]:
import plotly.express as px
import pandas as pd

# Sample data for US states and their populations
data = {
    'State': ['California', 'Texas', 'Florida', 'New York', 'Pennsylvania', 'Illinois', 'Ohio', 'Georgia', 'North Carolina', 'Michigan',
              'New Jersey', 'Virginia', 'Washington', 'Arizona', 'Massachusetts', 'Tennessee', 'Indiana', 'Missouri', 'Maryland', 'Wisconsin'],
    'Population': [39538223, 29145505, 21538187, 20201249, 13002700, 12822739, 11799448, 10711908, 10439388, 10077331,
                    9294493, 8631393, 7693612, 7151502, 7029917, 6910840, 6785528, 6160281, 6185278, 5893718]
}

df = pd.DataFrame(data)

# Sort the DataFrame based on Population in descending order
df = df.sort_values(by='Population', ascending=False)

# Create the bar chart
fig = px.bar(df, x='State', y='Population', title='Population of Top 20 US States', text='Population')
fig.update_traces(texttemplate='%{text:,}', textposition='outside')
fig.update_layout(xaxis_title='State', yaxis_title='Population', yaxis=dict(type='linear'))
fig.show()


From the above 2 visualizations we can see some interesting trends 

- Even though Cali is the highest populated, it does not have the most number of patients
- New York is 4th in terms of total population but 10th in the number of patients 

Geospatial representation 

In [20]:
import pandas as pd
import plotly.express as px

# Read data
df_institute = pd.read_excel('pr1.xlsx', sheet_name='Accessioned_By_Institute')

# Calculate total patients for each institute
df_institute['Total_Patients'] = df_institute.iloc[:, 5:].sum(axis=1)

# Aggregate total patients by state
state_patient_counts = df_institute.groupby('State')['Total_Patients'].sum().reset_index()

# Create interactive choropleth map
fig = px.choropleth(state_patient_counts, 
                    locations='State', 
                    color='Total_Patients',
                    locationmode="USA-states", 
                    scope="usa",
                    title='Total Number of Patients Across All States',
                    color_continuous_scale="Viridis",
                    hover_name='State',
                    hover_data={'State': False, 'Total_Patients': True})
fig.update_geos(fitbounds="locations")
fig.show()

'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'

# Gender related trends

In [21]:
import pandas as pd
import plotly.express as px

# Load the data
df_patient_results = pd.read_excel("pr1.xlsx", sheet_name="Patient_Results")

# Create a bar plot for gender distribution
fig_gender = px.bar(df_patient_results['Sex'].value_counts().reset_index(), 
                    x='index', 
                    y='Sex', 
                    labels={'index': 'Gender', 'Sex': 'Count'},
                    title='Gender Distribution of Patients')
fig_gender.show()


In [22]:
# Prepare the data for geospatial visualization
state_gender_distribution = df_patient_results.groupby(['Institution_ST', 'Sex']).size().unstack().reset_index()
state_gender_distribution = state_gender_distribution.fillna(0)

# Create a choropleth map for males
fig_male = px.choropleth(state_gender_distribution, 
                         locations='Institution_ST', 
                         color='Male',
                         locationmode="USA-states", 
                         scope="usa",
                         title='Distribution of Male Patients Across States')
fig_male.show()

# Create a choropleth map for females
fig_female = px.choropleth(state_gender_distribution, 
                           locations='Institution_ST', 
                           color='Female',
                           locationmode="USA-states", 
                           scope="usa",
                           title='Distribution of Female Patients Across States')
fig_female.show()

'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'

- The distribution of male and females across the states is the same 
- Even the ratio of males to females across the states is somewhat similar to the ratio of totoal males to total females 